In [ ]:
# versions: pandas==2.0.3, nibabel==5.2.1, numpy==1.24.4, scipy==1.8.0

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import numpy.linalg as npl
from scipy.ndimage import map_coordinates


In [ ]:
DATASET_TRAIN="./data/CN+MCI_BL+VOL_train.csv"
DATASET_TEST="./data/CN+MCI_BL+VOL_test.csv"
FASTSURFER_OUTPUT_DIR="./data/ADNI/output_fastsurfer/"
VBM_RESULTS_DIR="./results/VBM/ADNI/"
VBM_RESULTS_REGION_FILE="./results/VBM_Results_Texture_all_not_filtered_new.csv"

In [ ]:
freesurferMapping=pd.read_table("./data/fastsurferMappingReduced.csv",sep=",")


In [ ]:
pvalue_img=nib.load(VBM_RESULTS_DIR+"/VBM_p_value_results_all_fdr.nii.gz")  
pvalue_data=np.asanyarray(pvalue_img.dataobj)
img=pvalue_data


In [ ]:
data_aseg=pd.read_table(FASTSURFER_OUTPUT_DIR+"aseg_stats.txt")


data_aseg=data_aseg.rename({"Measure:volume":"filename"},axis=1)
data_aseg["IMAGEUID"]=data_aseg.filename.str.split("_I").str[-1].str[:-1]
data_aseg=data_aseg.astype({"IMAGEUID": "str"})
data_aseg=data_aseg.filter(["filename","IMAGEUID"])
paths=FASTSURFER_OUTPUT_DIR+data_aseg.filename+"/mri/mri/mwp1norm.nii"
data_aseg["filename_mwp1"]=paths

paths=FASTSURFER_OUTPUT_DIR+data_aseg.filename+"/mri/mri/iy_norm.nii"

data_aseg["filename_iy"]=paths
paths=FASTSURFER_OUTPUT_DIR+data_aseg.filename+"/mri/aparc.DKTatlas+aseg.deep.mgz.nii.gz"

data_aseg["filename_aparc"]=paths

In [ ]:
df_ges=pd.read_csv(DATASET_TEST)
df_ges=df_ges.astype({"IMAGEUID": "str"})


In [ ]:
df_ges=pd.merge(df_ges,data_aseg,on="IMAGEUID",how="left")

In [ ]:
column_names = list(freesurferMapping["brain region"])
df = pd.DataFrame()

In [ ]:
for index,dft in df_ges.iterrows():
    print(index)
    deformations_img = nib.load(dft["filename_iy"])
    deformations_data = np.asanyarray(deformations_img.dataobj)
    deformations_data = np.squeeze(deformations_data)
    template_img = nib.load(dft["filename_mwp1"])
    template_data = np.asanyarray(template_img.dataobj)
    vox2vox_mapping = nib.affines.apply_affine(npl.inv(template_img.affine), deformations_data)
    for_map_coords = vox2vox_mapping.transpose(3, 0, 1, 2)
    subject_into_tpm = map_coordinates(img, for_map_coords)
    segmentation_img = nib.load(dft["filename_aparc"])
    segmentation_data = np.asanyarray(segmentation_img.dataobj)
    seg_Flattened=segmentation_data.flatten()
    res_flattened=subject_into_tpm.flatten()
    colors, counts = np.unique(seg_Flattened, axis=0, return_counts=True)
    summedValues=[0]*counts.shape[0]
    summedValues=np.asarray(summedValues)
    j=0
    for i in colors:
        summedValues[j]=res_flattened[seg_Flattened==i].sum()
        j+=1
    meanValues=summedValues/counts
    d = {"ID": colors, "meanValues": meanValues,"summedValues":summedValues, "counts":counts}
    dfSub = pd.DataFrame(data=d)
    dfSub=dfSub.sort_values("ID")
    merged_test=freesurferMapping[freesurferMapping.ID.isin(colors)]
    merged_test=merged_test.sort_values("ID")
    column_names = list(merged_test["brain region"])
    values=meanValues.tolist()
    df2 = pd.DataFrame([values],columns = column_names)
    df=pd.concat([df, df2], ignore_index=True)

In [ ]:
df.to_csv(VBM_RESULTS_REGION_FILE)